# K-Means Clustering — No-Framework Implementation

Building K-Means from scratch using only NumPy. This is the most educational implementation — we manually code every component of the algorithm.

**Dataset**: Dry Beans — 13,543 samples, 16 geometric features, 7 bean types.

## What We'll Build From Scratch
- **K-Means++ initialization** — smart centroid seeding (probability ∝ distance²)
- **Cluster assignment** — vectorized Euclidean distance via broadcasting
- **Centroid update** — mean of assigned samples + empty cluster handling
- **Lloyd's algorithm** — the iterative assign → update → converge loop
- **Multi-init wrapper** — n_init runs, keep best by inertia


In [1]:
import sys
import os
import numpy as np

# Add project root to path for utils
sys.path.insert(0, os.path.abspath('../..'))
from utils.data_loader import load_processed_data
from utils.performance import track_performance
from utils.visualization import (plot_elbow_curve, plot_silhouette_comparison,
                                  plot_silhouette_analysis, plot_convergence_curve)
from utils.results import save_results, add_result, print_comparison
from utils.metrics import inertia, silhouette_score, adjusted_rand_index

# Configuration
RANDOM_STATE = 113
K_RANGE = range(2, 13)       # Test K=2 through K=12
MAX_ITER = 300
TOL = 1e-4
N_INIT = 5                   # 5 random initializations, keep best by inertia
FRAMEWORK = 'No-Framework'

# Load preprocessed data
X_train, X_test, y_train, y_test, meta = load_processed_data('kmeans')

print("=" * 60)
print(f"K-MEANS — {FRAMEWORK}")
print("=" * 60)
print(f"Training: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
print(f"Test:     {X_test.shape[0]:,} samples")
print(f"Classes:  {meta['n_classes']} ({meta['class_names']})")

K-MEANS — No-Framework
Training: 10,834 samples, 16 features
Test:     2,709 samples
Classes:  7 (['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA'])


In [2]:
# Step 1: K-Means++ initalization

def kmeans_plus_plus_init(X, k, rng):
    """
    Select k initial centroids using k-means++ algo.

    First centroid is chosen randomly. Each subsequent centroid is 
    chosen with probability prorportional to D(x)^2 - the squared
    distance to the nearest existing centroid. This spreads centroids
    apart for faster, more reliable convergence.

    Args:
        X: training data (n_samples, n_featres)
        k: Number of clusters
        rng: np.random.Randomstate for reproducibility

    Returns:
        centroids: initial centroid positions (k, n_features)
    """
    n_samples, n_features = X.shape
    centroids = np.empty((k, n_features))

    # First centroid: random data point
    first_idx = rng.randint(0, n_samples)
    centroids[0] = X[first_idx]

    # Remaining centroids: weighted by squared distance
    for c in range(1, k):
        # Squared distance from each point to nearest existing centroid
        diff = X[:, np.newaxis, :] - centroids[np.newaxis, :c, :]
        sq_distances = np.sum(diff ** 2, axis=2)

        # Distance to nearest centroid for each point
        min_sq_dist = np.min(sq_distances, axis=1)

        # Convert to probabilities
        probs = min_sq_dist / min_sq_dist.sum()

        # Choose next centroid using weighted probability
        next_idx = rng.choice(n_samples, p=probs)
        centroids[c] = X[next_idx]

    return centroids

# Quick test
rng = np.random.RandomState(RANDOM_STATE)
test_centroids = kmeans_plus_plus_init(X_train, k=3, rng=rng)
print(f"Centroids shape: {test_centroids.shape}")
print(f"First centroid: {test_centroids[0][:5]}...")
print(f"Centroids are unique: {len(np.unique(test_centroids, axis=0)) == 3}")

Centroids shape: (3, 16)
First centroid: [ 0.1160314   0.49730625  0.79295167 -0.35469757  2.0429446 ]...
Centroids are unique: True


## Lloyd's Algorithm (Core K-Means Loop)

The iterative heart of K-Means. Each iteration has two steps:

1. **Assign** — For each data point, find the nearest centroid (Euclidean distance). This assigns every point to a cluster.
2. **Update** — Recompute each centroid as the mean of all points assigned to it.

Repeat until centroids stop moving (convergence) or max iterations reached.

**Convergence check:** Max centroid displacement < tolerance (1e-4). If no centroid moves more than this threshold, the algorithm has stabilized.


In [4]:
# Step 2: Assign clusters - find nearest centroid for each point

def assign_clusters(X, centroids):
    """
    Assign each sample to its nearest centroid using euclidean distance.

    Vectorized via broadcasting:
        X: (n_samples, n_features) -> (n_samples, 1, n_features)
        centroids: (k, n_features) -> (1, k, n_features)
        diff: (n_samples, k, n_features) - all pairwise differences
    
    Args:
        X: data points (n_samples, n_features)
        centroids: current centroid position (k, n_features)

    Returns:
        labels: cluster assignment for each sample (n_samples,)
        distances: distance from each sample to  its assigned centroid (n_samples,)
    """
    # Broadcasting: compute distance from every point to every centroid
    diff = X[:, np.newaxis, :] - centroids[np.newaxis, :, :]
    sq_distances = np.sum(diff ** 2, axis=2)    # (n_samples, k)

    # Each point gets assigned to the closest centroid
    labels = np.argmin(sq_distances, axis=1)

    # distance to assigned centroid (for inertia calculation)
    distances = sq_distances[np.arange(len(X)), labels]

    return labels, distances

# Quick test
test_labels, test_dists = assign_clusters(X_train[:10], test_centroids)
print(f"Labels shape: {test_labels.shape}")
print(f"Labels: {test_labels}")
print(f"Distances shape: {test_dists.shape}")
print(f"All distances positive: {np.all(test_dists >= 0)}")

Labels shape: (10,)
Labels: [0 0 0 0 0 2 0 2 2 2]
Distances shape: (10,)
All distances positive: True


In [6]:
# Step 3: Update centroids - recompute as mean of assigned samples

def update_centroids(X, labels, k, rng):
    """
    Recompute each centroid as the mean of its assigned samples.

    Edge case: if a cluster has no samples assigned (empty cluster),
    reinitalize it to a random data point. This prevents NaN centroids
    and keep all k clusters active.

    Args:
        X: data points (n_samples, n_features)
        labels: current cluster assignemnts (n_samples,)
        k: number of clusters
        rng: np.random.RandomState for empty cluster reinitialization

    Returns:
        centroids: updated centroid positions (k, n_features)
    """
    n_features = X.shape[1]
    centroids = np.empty((k, n_features))

    for c in range(k):
        mask = labels == c
        if np.sum(mask) > 0:
            centroids[c] = X[mask].mean(axis=0)
        else:
            # Empty cluster: reinitialize to random data point
            centroids[c] = X[rng.randint(0, len(X))]
    
    return centroids

# Quick test
rng = np.random.RandomState(RANDOM_STATE)
test_updated = update_centroids(X_train[:10], test_labels, k=3, rng=rng)
print(f"Updated centroids shape: {test_updated.shape}")
print(f"No NaN values: {not np.any(np.isnan(test_updated))}")

Updated centroids shape: (3, 16)
No NaN values: True


In [7]:
# Step 4: single run of lloyds algorithm

def kmeans_single_run(X, k, max_iter=300, tol=1e-4, rng=None):
    """
    One complete run of k-means: initialize, then iterate assign/update.

    Lloyds algorithm loop:
        1. initailize centroids (k-means++)
        2. assign each point to nearest centroid
        3. update centroids as cluster means
        4. check convergence: if max centroid shift < tol, stop
        5. repeat 2-4 until converged or max_iter reached
    
    Args:
        X: training data (n_samples, n_features)
        k: number of clusters
        max_iter: maximum iterations before stopping
        tol: convergence threshold - max centroid displacement
        rng: np.random.RandomState for reproducibility

    Returns:
        centroids: final centroid positions (k, n_features)
        labels: final cluster assignments (n_samples,)
        final_inertia: sum of squared distances to centroids
        n_iter: number of iterations completed
        inertia_history: inertia at each iteration (for convergence plot)
    """
    # initialize centroids using k-means++
    centroids = kmeans_plus_plus_init(X, k, rng)
    inertia_history = []

    for iteration in range(1, max_iter + 1):
        # assign step
        labels, distances = assign_clusters(X, centroids)
        current_inertia = np.sum(distances)
        inertia_history.append(current_inertia)

        # update step
        new_centroids = update_centroids(X, labels, k, rng)

        # convergence check: max centroid displacement
        shift = np.max(np.sqrt(np.sum((new_centroids - centroids) ** 2, axis=1)))
        centroids = new_centroids

        if shift < tol:
            break

    return centroids, labels, current_inertia, iteration, inertia_history

# Quick test with small k
rng = np.random.RandomState(RANDOM_STATE)
cents, labs, inert, n_it, hist = kmeans_single_run(X_train, k=3, rng=rng)
print(f"Converged in {n_it} iterations")
print(f"Final inertia: {inert:,.2f}")
print(f"Inertia history length: {len(hist)}")
print(f"Inertia decreased: {hist[0] > hist[-1]}")

Converged in 7 iterations
Final inertia: 75,557.90
Inertia history length: 7
Inertia decreased: True
